In [1]:
# reloading magic
%load_ext autoreload
%autoreload 2

In [2]:
log_file = None

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [4]:
import os
import sys
comm_path = os.path.normpath('C:/cygwin64/home/oliverlee/repos/bikesim/comm')
scripts_path = os.path.normpath('C:/cygwin64/home/oliverlee/repos/bikesim/scripts')
for d in (comm_path, scripts_path):
    if d not in sys.path:
        sys.path.append(d)
from parse_data import *

In [5]:
# get the most recent log file in the comm_dir if not specified
if log_file is None:
    logs = [f for f in os.listdir(comm_path)
            if f.startswith('log_') and os.path.isfile(os.path.join(comm_path, f))]
    logs.sort()
    log_file = os.path.join(comm_path, logs[-1])
print('reading from log file: {}'.format(log_file))

reading from log file: C:\cygwin64\home\oliverlee\repos\bikesim\comm\log_150617_162518_UTC


In [6]:
sensor, actuator = parse_log(log_file)

In [7]:
fig1, ax1, hd1 = plot_timeinfo(sensor)
#fig2, ax2, hd2 = plot_timeinfo(actuator, 0.1)
plt.show()

sensor time median = 0.010001 s, std = 0.006659


In [8]:
fix3, ax3 = plt.subplots()
ax3.plot(sensor.time, sensor.deltad)
ax3.set_xlabel('time [s]')
ax3.set_ylabel('deltad [rad/s]')
plt.show()
print('mean: {:0.6f}, std: {:0.6f}'.format(np.mean(sensor.deltad), np.std(sensor.deltad)))

mean: -0.001781, std: 0.000324
